 
- Each storage container comes with a set of Access keys that one can use to read and write data which are called as Storage Access Keys.
- They also come with a Shared Access Signature token or SAS Token that can be used to maintain the the access to the user at a more granular level than the Storage Access Keys.
- Service Principal can be assigned with particular permission to access the datalake and then credentials of this service princiapl can be used to access the datalake from databricks

The above mentioned methods to access the data lake container can be achecived from databricks using 
  - Session Scoped Authentication 
    - Usage of the above credentials is within the notebook.
    - This access would tied to that session i.e., until the notebook is detached from the cluster 
  
  - Cluster Scoped Authentication
    - Usage of the credentials at the cluster level and would be valid until the cluster is terminated

- Through Azure Active Directory (AAD)
  - AAD Passthrough Authentication: where the databricks cluster looks at the credentials of user and based on his roles assigned through the IAM( Identity Access Management) will be able to parse through the data the user has access to

1) Click on the hamburger icon on the top left corner of the console. </br>
2) Create a Resource and search for storage account. </br>
3) Select the subscription from teh dropdown for billing purposes. </br>
4) Select the resource group under which the storage account should be created for. </br>
5) Enter the nema of the storage container nad the region closest to you and also the redundacy method for maintaining a HA storage account.</br>
    - there are four types of redundancies to choose from:
      - LRS (Local Redundant Storage)
      - ZRS (Zone Redundant Storage)
      - GRS (Geo Redundant Storage)
      - GZRS (Geo Zone Redundant Storage)
6) Click next on to Advanced settings and to enable the Datalake storage (ADLS v2) capability of the storage container select the tick mark the option to enable Hierarchical Namespace.
7) Leaving everything as it is click on review and create the storage account 

- Each storage account comes two Access Keys
  - Has full access to the storage account (Super User).
  - Recommended secure using Azure Key Vault.
  - If Key is compromissed/lost they can rotated or regenerated.

 setting up a spark configuration on Azure Databricks using the below command would help databeicks access the data stored in the data container
 
 spark.conf.set("fs.azure.account.key.\<storage-account\>.dfs.core.windows.net","\<access-key\>") </br>
 spark.conf.set(config.endpoint,access_key)

 - Azure Recommends to use **Azure Blob File System (ABFS)** driver to access data in the containers rather than the https protocol 

 URI Syntax 
 - abfss://container@storage_account_name.dfs.core.windows.net/folder_path/file_name 
  - Ex. abfss://demo@f1azda.dfs.core.windows.net/circuits.csv
  - Ex. abfss://demo@f1azda.dfs.core.windows.net/test/circuits.csv

Once we have the URI we can use the dbutils.fs.ls command along with the URI to access the data within the storage container

In [0]:
Provides more fine grained access to the data in the storage accounts by:
-   Restricting access to only specific type of files or only access specific containers in the storage account.
-   Allowing specific permissions to the user like the user is only allowed to read data within the files but not update/write.
-   Restricting the users to access the storage accounts only at a given time period of the day.
-   Limiting the access to the data to a range or specific IP addresses.
-   Best Practice to share SAS tokes with External clients who dont have direct access to the Environment.


Service Priniciples are similar to users which can be defined in the Azure Active Directory and then be assigned RBAC roles to assign permissions/ level of access to various Azure Services 
- Recommended approach to give access to Services during automation processes such as in CI/CD pipelines or automated job runs 

Above Discussed access methods are valid only for a given session that is valid only till the notebook is detached from from teh cluster or until the cluster automatically terminates.
So we can assign variables during cluster startup so that the authentication is done during cluster initialization and all the notebooks attahced to the cluster or users accessing the cluster can access the resources (for ex. directly access the data in the data lake) without any setup in the notebook.

But this approach is not a good approach in the case of a large organisation as individual team should have thier cluster launched as the application for each team will be different and this architecture would difficult to manage as the team/ organisation grows.

Databricks --> Secret Scopes and in Azure --> Azure Key Vault

Secret Scopes are a way through which credentials can be stored in databricks securily by referencing them in clusters, notebooks and jobs
- Two types of Scopes 
    - Databricks backed Secret Scope --> This is a solution which is backed and owned by Databricks and can be accessed by Databricks CLI 
    - Azure Key Vault backed Secret Scope (Recommended approach while using Databricks on Azure)

## Linking Azure Key Vault to Databricks 

- Create a Azure Key vault Resource --> Link it to the Databricks backed Scope --> reference the secrets using the databricks secrets utility **dbutils.secrets.get**

## Process 

- Step 1: Copy the token values from the azure container (Access key token, SAS Token)
- Step 2: Go to Azure key Vault and create a new Vault
- Step 3: Within the newly created Vault --> go to Secrets option in the side menu bar --> import/generate a new secret (top menu bar) --> Give the secret a name and secret value (Like a Key Value pair).
- Step 4: In the key Vault --> Go to Properties under settings in the side menu --> make a note of the **Vault URI** and **Resource ID**.
- Step 5: In the databricks homepage modify the URL to include **#secrets/createScope** at the end and also make sure that you are not there on onboarding page i.e., in the URI after **azuredatabtricks.net/onboarding?o=** --> **incorrect** and it should be **azuredatabtricks.net/?o=**.
- Step 6: In the create secret Scope menu give a name to the secret scope and then also provide the details such as **Azure Vault URI** and **Resource ID** to link the Azure Key Vault to the Databricks Secret Scope feature.
- Step 7a: To get the list of Scopes available in the Secret Scope database --> dbutils.secrets.listScopes()
- Step 7b: To get the list of Secrets within the scope --> dbutils.secrets.list(scope = "\<Scope Name>")
- Step 7c: To get the information from a particular scope and an element in the list --> dbutils.secrets.get(scope = "\<Scope Name>", key = "\<Key Value>")

- Comprised of **driver and Worker nodes** and a **cluster manager node**
  -  The Driver is repsonsible for setting up the sparkContext which acts as an Entry point into the spark cluster 
  -  Once the program is submitted the logical plan for execution is created at the driver node.
  -  the logical plan is made until by the **DAG Scheduler** until an action keyword is called and based on the differnet transformations till then the plan is then split into stages which consist of different tasks.
  -  Based on the dataset it is operating the driver node does negotiates with the cluster manager on allocation of resources(worker nodes) to complete the tasks.
  -  The Cluster manager spins up the worker nodes with the help of the **Task Scheduler** and communicates the job to be performed by them.
  -  The status of the jobs are reported periodically to the cluster manager and incase of the a worker node failure the cluster manager will spin up a new cluster and reassign the task the lost node as assigned.

References:
- https://www.interviewbit.com/blog/apache-spark-architecture/
- https://www.databricks.com/glossary/catalyst-optimizer
- https://www.analyticsvidhya.com/blog/2021/08/understand-the-internal-working-of-apache-spark/
- https://stackoverflow.com/questions/24909958/spark-on-yarn-concept-understanding/38598830#38598830


- Can use the %run command to run another notebook from within another one and use the variables or functions from the 
- For Example Syntax can be like: </br>
  **%run ../path/to/file**

In [0]:
dbutils.widgets.<option>

dbutils.widgets.help --> gets us the manual on how to use the various other methods which can be used to configure widgets. 
dbutlis.widgets.text("param_name","default_val") --> creates a text box on top of the page on what parameter value can be passed through.
dbutils.widgets.get("param_name") --> get the value entered in the parameter above created to bring it into the context of the workbook.

and more...


- Hive Metastore is a registry which stores information about the metadata of the files in a tablular manner which includes details such as location of the file, file extension etc.

- Apache Hive provides a metastore for Spark to use while query.

#### Spark Implementation

- When SQL query is querying a table from a spark context it first checks for the metadata in the hive catalog and if it is present the data 

- Types of tables that can be created using the spark sql are Managed and external where
  - Managed table is a kind of table where the metadata and table data is handled by Spark SQL itself 
  - External table is a kind of table where the only the metadata is handled by spark and data is handled by the user

- **NOTE: When a managed table is deleted by spark the metadata and the data is deleted and when an external table is deleted, metadata only gets deleted where as the data still remains as is**


## Delta Lake
  - This is a datalake with ACID transaction controls which enables delta table to ingest streaming data along with the batch load processing.

  - There are three stages of transformation which are:
    - raw data storage (Bronze Zone)
    - Pre processed data storage (Silver Zone)
    - Aggregated data storage for BI reporting (Gold Zone)

#### UPDATE

1) How to update an existing delta table using SQL 
    - pretty straight forward:
    - UPDATE TBL_NM SET COL_NAME = OPERATION WHERE CONDITION 
    - Eg:- UPDATE RESULTS_MANAGED SET POINTS = 11 - POSITION WHERE POSITION < 11
2) How to update an existing delta table using Python 
    - import statement --> from delta.tables import DeltaTable
    - delta_table = DeltaTable.forPath("/mnt/f1azda/demo/results_managed")
    - delta_table.update("position<11",{"points":"21-position"})

#### DELETE 

4) Delete data from delta table using SQL
    - DELETE FROM tbl_nm where condition

5) Delete data from delta table using python \
    from delta.tables import DeltaTable \
    deltatable = DeltaTable.forPath(spark,"/mnt/f1azda/demo/result_managed") \
    deltatable.delete("position > 10")

#### MERGE/ UPSERT(Update or Insert)

6) SQL Syntax \
    -  MERGE INTO TGT_TBL \
       USING SRC_TBL \
       ON JOIN CONDITION \
       WHEN MATCHED THEN \
       UPDATE SET TGT_TBL.COL = SRC_TBL.COL \
       WHEN NOT MATCHED THEN \
       INSERT (TGT_TBL COLS) VALUES (SRC_TBL COLS) \
7) PySpark Syntax: \
    - from delta.tables import DeltaTable\
      from pyspark.sql.functions import * as f \
      _loading the target table_ \
      delta_table = DeltaTable.forPath(spark,"path_to_storage folder") \
      _defining the merge function_ \
      delta_table.alias("TGT").merge(source_tbl.alias("SRC"),"TGT.col_name = SRC.col_name")\
      .whenMatchedUpdate(set = {"TGT.col2" : "SRC.col2",\
                                "TGT.col3" : "SRC.col3"})\
      .whenNotMatchedInsert(values = {"TGT.col1" : "SRC.col1", \
                                      "TGT.col1" : "SRC.col1"}) \
      .execute()
    
#### TIME TRAVEL

3) Time travel
- SQL Syntax
    - RESTORE TABLE tbl_nm TO TIMESTAMP AS OF of "yyyy-mm-dd hh:mm:ss.ms"
    - To know the history of changes made to a delta table we can use:
    -  DESCRIBE HISTORY tbl_name
- To get the table data from a particular point in time or a specific version.

    - SELECT * FROM tbl_nm VERSION AS OF 1;
    - SELECT * FROM tbl_nm TIMESTAMP AS OF "2024-03-20T21:49:09.000+00:00"

- Python Syntax
    - Format to get the older version of the dataframe from the file using the version.
    df_old = spark.read.format("delta").option("versionAsOf",1).load("/mnt/f1azda/demo/drivers_day_merge")
    - Format to get the older version of the dataframe from the file using the timestamp.
    df_old = spark.read.format("delta").option("timeStampAsOf",1).load("/mnt/f1azda/demo/drivers_day_merge")

#### VACUUM METHOD TO SATISY GDPR CONSTRAINTS 
    - Vacuum is the method to delete the history of all the data and the latest data in the delta lake
    - SQL Syntax
        - VACUUM tbl_nm RETAIN n HOURS --> default is 168 HOURS that is 7 days
    - To change the default to lesser than 7 days we have to set the spark parameter
        - SET spark.databricks.delta.rententionDurationCheck.enabled = false

#### Sccenario --> Data has been corrupted and it needs to be brought back to its original state
- SQL syntax (using MERGE INTO )
    - MERGE INTO TGT_TBL \
      USING SRC \
      ON TGT.COL = SRC.COL\
      WHEN NOT MATCHED \
      INSERT * 



#### Transaction log for a delta table 
1) For each operation on the table (Insert, Update, Delete) a new parquet file is created in the storage location and a corresponding 
   log transactionfile in the form of a json file.
2) a. when a table is created it first creates a parquet file in teh given location of the database. \
   b. Also a file in the transaction log is created with vital information against the processs. \
   c. When more and more data is being generated there will be a transaction checkpoint to which the process can refer to ignoring 
      all the individaul files that represent sinlgle transactions s that happened on the table before the checkpoint. \s

3) Transaction table content when a table is created is pretty straight forward it contains the following:
  - Transaction commit information:
    - timestamp: when the create table statment was commited.
    - Username: Email of the account responsible for the action.
    - Operation: Type of CRUD operation performed (WRITE). 
    - a few other details such as notebook information, cluster information, transaction id
  - etadata
    - createdTime: Created time stamp
    - paritionColumns (list): if the table is to be partitoned by any columnm it would be mentioned here 
    - schemaString: type of the table which is \
        - data type of table :"struct"
        - field (list of dictionaries):contains information about the fields in the table such as name, type, nullable, metadata of a given column in the table 

4) Transaction table content when a record in a table is added :
   -Transaction commit info
    - timestamp: when the create table statment was commited.
    - Username: Email of the account responsible for the action.
    - Operation: Type of CRUD operation performed (CREATE TABLE). 
    - a few other details such as notebook information, cluster information, transaction id
  - add (as a new parquet file is created when ever an operation is done on a delta table)
    - path: "name of the file being referred to"
    - dataChange: Boolean object stating if there is any change in the underlying data of the file.
    - numstats (numerical stats): suggesting min and max values in each columns, numOfRecords, and also number of null values in each column is mentioned.

5) Transaction table content when a record in a table is deleted :
   -Transaction commit info
    - timestamp: when the create table statment was commited.
    - Username: Email of the account responsible for the action.
    - Operation: Type of CRUD operation performed (DELETE). 
    - a few other details such as notebook information, cluster information, transaction id
    - OperationMetrics are mentioned such as number of removed files, number of removed records, time taken to scan the table for finding the records and also time taken to rewrite the information.  
  - remove 
    - path: "name of the file being referred to"
    - dataChange: Boolean object stating if there is any change in the underlying data of the file.
    - Deletion_timestamp --> epoch tome of when a file is deleted.  

6) There is also a .crc --> Checksum Redundancy Check file which can be used to check if the data is a match. 
      


In [0]:
import hashlib

def mask_sensitive_data(input_data, field_to_mask):
    """
    Mask the specified field in the input data using a hash function.

    Parameters:
    - input_data: Dictionary containing sensitive information.
    - field_to_mask: Name of the field to be masked.

    Returns:
    - Masked data.
    """
    masked_data = input_data.copy()  # Create a copy to avoid modifying the original data

    if field_to_mask in masked_data:
        original_value = masked_data[field_to_mask]

        # Using SHA-256 hash for masking (you can choose a different method based on your requirements)
        print(original_value)
        hash_object = hashlib.md5(original_value.encode())
        print(hash_object)
        masked_value = hash_object.hexdigest()
        print(masked_value)

        masked_data[field_to_mask] = masked_value

    return masked_data

# Example Usage:
sensitive_data = {
    "patient_id": "12345",
    "patient_name": "John Doe",
    "ssn": "123-45-6789",
    "medical_history": "Sensitive information here",
}

field_to_mask = "patient_name"

masked_data = mask_sensitive_data(sensitive_data, field_to_mask)
# print("Original Data:", sensitive_data)
# print("Masked Data:", masked_data)

John Doe
<md5 _hashlib.HASH object @ 0x7f32d42d6b50>
4c2a904bafba06591225113ad17b5cec
